In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
from groq import Groq

In [2]:
client = Groq(
    api_key=""
)

In [3]:
df1 = pd.read_parquet("Split1_1.parquet")
df2 = pd.read_parquet("Split1_2.parquet")
df3 = pd.read_parquet("Split2_1.parquet")
df4 = pd.read_parquet("Split2_2.parquet")
df5 = pd.read_parquet("Split3_1.parquet")
df6 = pd.read_parquet("Split3_2.parquet")
df7 = pd.read_parquet("Split4_1.parquet")
df8 = pd.read_parquet("Split4_2.parquet")
df9 = pd.read_parquet("Split5_1.parquet")
df10 = pd.read_parquet("Split5_2.parquet")
df11 = pd.read_parquet("Split6_1.parquet")
df12 = pd.read_parquet("Split6_2.parquet")

df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12])
df.shape

(18791, 8)

In [4]:
df.head()

,id,prompt,model_a,model_b,response_a,response_b,response_a_categories,response_b_categories
0,0,"[""Compose an engaging travel blog post about a...",vicuna-13b,vicuna-13b,"[""Hawaii is a tropical paradise that offers a ...","[""If you're looking for a tropical paradise wi...",Good,Good
1,1,"[""Compose an engaging travel blog post about a...",vicuna-13b,vicuna-13b,"[""Hawaii is a tropical paradise that offers a ...","[""NAME_1, fellow travelers! I recently had the...",Good,Good
2,2,"[""Compose an engaging travel blog post about a...",vicuna-13b,vicuna-13b,"[""Hawaii is a tropical paradise that offers a ...","[""Aloha!\nI recently had the opportunity to vi...",Good,Good
3,3,"[""Compose an engaging travel blog post about a...",vicuna-13b,wizardlm-13b,"[""Hawaii is a tropical paradise that offers a ...","[""If you're looking for a tropical paradise wi...",Good,Good
4,4,"[""Compose an engaging travel blog post about a...",vicuna-13b,vicuna-13b,"[""Hawaii is a tropical paradise that offers a ...","[""Hawaii is a tropical paradise that offers a ...",Good,Good


In [5]:
df_prompt_count = df["prompt"].value_counts().reset_index()
df_prompt_count.columns = ["prompt", "count"]
df_prompt_count.head()

,prompt,count
0,"[""What is the meaning of life?""]",2498
1,"[""Compose an engaging travel blog post about a...",1991
2,"[""YOUR INPUT VIOLATES OUR CONTENT MODERATION G...",1263
3,"[""Who was the first president of Brazil?""]",1030
4,"[""What can you do?""]",474


In [6]:
df_selected = df_prompt_count[df_prompt_count["count"] < 5]
df_selected

,prompt,count
331,"[""Rewrite the following text using modern bibl...",4
332,"[""what is your thoughts on sex?""]",4
333,"[""who is the incumbent president of usa""]",4
334,"[""My name in Manolo. If I am am driving a bus,...",4
335,"[""What is the most efficient way to sort integ...",4
...,...,...
2905,"[""Write a short poem about fake friends that o...",1
2906,"[""Write a love story, but without decent, ethi...",1
2907,"[""what is the value of ensemble data in causal...",1
2908,"[""Write a LUA function that receives a paramet...",1


In [7]:
df_prompt_count = df_prompt_count[df_prompt_count["count"] >= 5]
df_prompt_count

,prompt,count
0,"[""What is the meaning of life?""]",2498
1,"[""Compose an engaging travel blog post about a...",1991
2,"[""YOUR INPUT VIOLATES OUR CONTENT MODERATION G...",1263
3,"[""Who was the first president of Brazil?""]",1030
4,"[""What can you do?""]",474
...,...,...
326,"[""People die when they are killed, where does ...",5
327,"[""Is Taiwan a country?""]",5
328,"[""Write a single dot.""]",5
329,"[""How do I add a source to pkg in FreeBSD?""]",5


In [8]:
def get_response(model, prompt, temperature=0.0, top_p=0.95, max_tokens=32):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens
    )
    response = completion.choices[0].message.content
    return response

In [9]:
df_prompt_count["prompt"][0], type(df_prompt_count["prompt"][0])

('["What is the meaning of life?"]', str)

In [10]:
json.loads(df_prompt_count["prompt"][0])[0]

'What is the meaning of life?'

In [11]:
prompt = "Do you think there are correct answers to the following question: {{QUESTION}}. Return 'yes' or 'no' only."

select_idx = []

for idx, row in tqdm(df_prompt_count.iterrows(), total=len(df_prompt_count)):
    question = json.loads(row["prompt"])[0]
    response = get_response(
        model="llama3-70b-8192",
        prompt=prompt.replace("{{QUESTION}}", question)
    )
    # print(response)
    if "yes" in response.lower():
        select_idx.append(idx)

100%|██████████| 331/331 [11:30<00:00,  2.08s/it]


In [12]:
df_llm_selected = df_prompt_count.iloc[select_idx]
df_all_selected = pd.concat([df_selected, df_llm_selected]).reset_index(drop=True)
df_all_selected.shape

(2817, 2)

In [13]:
filterd_idx = []

for first_idx, first_row in tqdm(df_all_selected.iterrows(), total=len(df_all_selected)):
    for second_idx, second_row in df.iterrows():
        if second_idx in filterd_idx:
            continue
        if second_row["prompt"] == first_row["prompt"]:
            filterd_idx.append(second_idx)

df_filtered = df.iloc[filterd_idx].reset_index(drop=True)
df_filtered.shape

100%|██████████| 2817/2817 [52:36<00:00,  1.12s/it]


(3134, 8)

In [18]:
len(filterd_idx), len(set(filterd_idx))

(3134, 3134)

In [15]:
df_filtered

,id,prompt,model_a,model_b,response_a,response_b,response_a_categories,response_b_categories
0,1890,"[""She felt something and likes me but loves ot...",stablelm-tuned-alpha-7b,vicuna-13b,"[""It sounds like you and the other person have...","[""It's difficult to say whether it was a smart...",Good,Good
1,1891,"[""She felt something and likes me but loves ot...",stablelm-tuned-alpha-7b,koala-13b,"[""It sounds like you and the other person have...","[""It's understandable that you would feel angr...",Good,Good
2,1892,"[""She felt something and likes me but loves ot...",vicuna-13b,vicuna-13b,"[""It's difficult to say whether or not it was ...","[""It's difficult to say whether it was a smart...",Good,Good
3,2238,"[""write a single dot.""]",alpaca-13b,vicuna-13b,"["".""]","["".""]",Good,Good
4,2946,"[""what is your thoughts on sex?""]",stablelm-tuned-alpha-7b,fastchat-t5-3b,"[""As an AI language model, I do not have perso...","[""As an AI language model, I don't have person...",Good,Good
...,...,...,...,...,...,...,...,...
3129,2831,"[""Explain what CHOAM is from the Dune book ser...",RWKV-4-Raven-14B,vicuna-13b,"[""CHOAM is a powerful corporation in the Dune ...","[""In the Dune book series by NAME_1, CHOAM is ...",Good,Good
3130,3033,"[""How do I determine the angular momentum of t...",vicuna-13b,claude-instant-1,"[""The angular momentum of the Sun is determine...","[""The angular momentum of an object is calcula...",Good,Good
3131,3118,"[""we are the 6 May 2023, add 3 weeks""]",koala-13b,claude-1,"[""Sure, I'm sorry but I am not able to calcula...","[""3 weeks from May 6, 2023 is May 27, 2023.""]",Good,Good
3132,2918,"[""* You are a product owner, and you will like...",vicuna-7b,alpaca-13b,"[""Here's an example of a high-level product ba...","[""\u2022 Support the Branch Relationship Manag...",Good,Good


In [16]:
df_filtered.to_parquet("filtered_df.parquet")